# Projeto de Estatística II
## Parte I - Regressão Linear

### Alexsander Vieira

Importando as bibliotecas do projeto.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from   sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler 
from   sklearn.metrics import r2_score
from scipy.stats import normaltest
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
import os

In [ ]:
df = pd.read_csv('cancer_EUA.csv', sep=';', decimal=',')

df.head()

In [ ]:
df.describe()

Analise da porcentagem de valores nulos em relação a quantidade de valores do dataset

In [ ]:
nulos = df.isnull().sum()

nulos_porcentagem = nulos/df.shape[0]

nulos_porcentagem

Analisando e excluindo quais features estão com mais de 50% de valores nulos

In [ ]:
nulos_porcentagem[nulos_porcentagem>0.5]

In [ ]:
df.drop('PctSomeCol18_24', axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
nulos = df.isnull().sum()

nulos_porcentagem = nulos/df.shape[0]

nulos_porcentagem

In [ ]:
nulos_porcentagem[nulos_porcentagem>0.5]

### Analises sob o TARGET_deathRate

In [ ]:
plt.title('Distribuição da taxa de mortalidade')
sns.histplot(data=df, x="TARGET_deathRate", kde=True)

In [ ]:

df['TARGET_deathRate'].skew()

In [ ]:
correlation = df.corr(method='spearman')

In [ ]:
correlation['TARGET_deathRate']

In [ ]:
corr_descending = correlation['TARGET_deathRate'].abs().sort_values(ascending=False)

corr_descending

In [ ]:
index = corr_descending[corr_descending>0.45].index

index

In [ ]:
count = 0
for i in index:
    plt.title('Distribuição da feature ' + index[count])
    sns.histplot(data=df[i], kde=True)
    plt.show()
    count +=1

In [ ]:
df['medIncome'].skew()

## PRIMEIRAS RESPOSTAS E CONCLUSÕES:

### 1 - A distribuição se mostrou assimétrica a direita por possuir skewness > 0.

### 2 - A variável escolhida para as análises que mais encaixou nas minhas análises é a medIncome

In [ ]:
plt.figure(figsize = (10,5))
plt.scatter(
    df['medIncome'], 
    df['TARGET_deathRate'], 
    c='blue')
plt.xlabel("TARGET_deathRate")
plt.ylabel("medIncome")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121); df.boxplot('medIncome')
plt.show()

In [ ]:
Q1_medIncome = df['TARGET_deathRate'].quantile(0.25)
Q3_medIncome = df['TARGET_deathRate'].quantile(0.75)
IQR_medIncome = Q3_medIncome - Q1_medIncome #distância entre quartis
print('{}: {}'.format("IQR TARGET_deathRate: ", IQR_medIncome))

In [ ]:
dataout_iqr_mask_TDR = (df['TARGET_deathRate'] < Q1_medIncome - (IQR_medIncome*1.5)) | (df['TARGET_deathRate'] > Q3_medIncome + (IQR_medIncome*1.5))
df[dataout_iqr_mask_TDR].sort_values(by='TARGET_deathRate')
print(df[dataout_iqr_mask_TDR].sort_values(by='TARGET_deathRate'))

In [ ]:
datain_iqr_mask_TDR = (df['TARGET_deathRate'] >= Q1_medIncome - (IQR_medIncome*1.5)) & (df['TARGET_deathRate'] <= Q3_medIncome + (IQR_medIncome*1.5))
df_iqr_clean_TDR = df[datain_iqr_mask_TDR]

sns.jointplot(data=df_iqr_clean_TDR, x='medIncome', y='TARGET_deathRate')
plt.show()


plt.figure(figsize=(10,5))
plt.subplot(121); df_iqr_clean_TDR.boxplot('medIncome')
plt.show()

In [ ]:
Q1_medIncome = df['medIncome'].quantile(0.25)
Q3_medIncome = df['medIncome'].quantile(0.75)
IQR_medIncome = Q3_medIncome - Q1_medIncome
print('{}: {}'.format("IQR medIncome: ", IQR_medIncome))

dataout_iqr_mask_medIncome = (df['medIncome'] < Q1_medIncome - (IQR_medIncome*1.5)) | (df['medIncome'] > Q3_medIncome + (IQR_medIncome*1.5))
df[dataout_iqr_mask_medIncome].sort_values(by='medIncome')
print(df[dataout_iqr_mask_medIncome].sort_values(by='medIncome'))

datain_iqr_mask_medIncome = (df['medIncome'] >= Q1_medIncome - (IQR_medIncome*1.5)) & (df['medIncome'] <= Q3_medIncome + (IQR_medIncome*1.5))
df_iqr_clean_medIncome = df[datain_iqr_mask_medIncome]

sns.jointplot(data=df_iqr_clean_medIncome, x='medIncome', y='TARGET_deathRate')
plt.show()

plt.figure(figsize=(10,5))
plt.subplot(121); df_iqr_clean_medIncome.boxplot('medIncome').set_title('Boxplot da variável medIncome após retirada dos outliers')
plt.show()

In [ ]:
df = df_iqr_clean_medIncome
df_clean = df
df_clean

In [ ]:
plt.figure(figsize=(20,20))
plt.title('Mapa de Calor das variáveis')
sns.heatmap(df_clean.corr(),annot=True, cmap="YlGnBu", linewidths=1, cbar_kws={"orientation": "horizontal"})

In [ ]:
X = df_clean['medIncome'].values.reshape(-1,1)
y = df_clean['TARGET_deathRate'].values.reshape(-1,1)


reg = LinearRegression()
reg.fit(X, y)


print("Mortes = {:.5} + {:.5}X".format(reg.intercept_[0], reg.coef_[0][0]))


In [ ]:
f_previsaoes = reg.predict(X)


plt.figure(figsize = (10,5))
plt.scatter(
    df['medIncome'], 
    df['TARGET_deathRate'], 
    c='blue')


plt.plot(
    df['medIncome'],
    f_previsaoes,
    c='red',
    linewidth=1
)

plt.xlabel(" medIncome ")
plt.ylabel(" TARGET_deathRate ")
plt.show()

In [ ]:
X = df_clean['medIncome']
y = df_clean['TARGET_deathRate']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
reg_lin = LinearRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.shape, y.shape
X_test.shape, y_test.shape
reg_lin.fit(X_train.values.reshape(-1,1),y_train)

In [ ]:
reg_lin.intercept_

In [ ]:
reg_lin.coef_[0]

In [ ]:
plt.figure(figsize=(15, 6))

# scatterplot da feature pelo target
sns.scatterplot(data=df_clean, x="medIncome", y="TARGET_deathRate")

# plot da reta do modelo
x = np.linspace(400,1200)
y = reg_lin.intercept_ + reg_lin.coef_[0]*x
plt.plot(x, y, color="red")

In [ ]:
y_pred = reg_lin.predict(X_test.values.reshape(-1,1))

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
residuo = y_test - y_pred

residuo

In [ ]:
residuo.describe()

In [ ]:
x = np.linspace(100, 250, 100000)
y = x
plt.plot(x, y, color="red")

sns.scatterplot(x=y_test, y=y_pred)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
test_stat, p = normaltest(y_test - y_pred)

print("Estatística de teste:", test_stat)
print("\np-value:", p)

# interpretando o teste
# nível de significância de 5%
alpha = 0.05

if p > alpha:
    print("\nFalha em rejeitar H_0: amostra parece ser normal")
else:
    print("\nRejeição da H_0: amostra não é normal")

print("Estatística de teste:", test_stat)
print()

In [ ]:
metrics.mean_absolute_error(y_test, y_pred)

In [ ]:
metrics.mean_squared_error(y_test, y_pred)

In [ ]:
metrics.mean_squared_error(y_test, y_pred, squared=False)

## Mais respostas e conclusões:

### 3 - R² = 0,179 indicando que apenas 17,9% do comportamento da variável TARGET_deathRate é explicada pela variável medIncome.

In [ ]:

df_clean.info()

In [ ]:
df_clean.select_dtypes(include=[np.number]).info()

In [ ]:
df_clean = df_clean.select_dtypes(include=[np.number])

df_clean = df_clean.dropna(axis="columns", how="any")

df_clean.select_dtypes(include=[np.number]).isnull().sum()

In [ ]:
X = df_clean.drop(columns=["TARGET_deathRate"])
y = df_clean['TARGET_deathRate']

In [ ]:
# etapas da modelagem

# 1)
estimador = LinearRegression()

# 2)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# 3)
modelo = estimador.fit(X_treino, y_treino)

print("Intercepto:", modelo.intercept_)
print("Coeficientes angulares:", modelo.coef_)

print("\n####################################################\n")

# 4)
predictions = modelo.predict(X_teste)

plt.title("valor predito vs valor real")
sns.scatterplot(x=predictions, y=y_teste)
plt.plot(np.linspace(100, 250, 1000), np.linspace(100, 250, 1000), color="red")
plt.show()

plt.title("Distribuição dos resíduos")
sns.histplot(y_teste-predictions, kde=True)
plt.show()

print("\n####################################################\n")

print("\nMétricas de avaliação (dados de teste):\n")

# 5)
from sklearn import metrics

print('R^2:', metrics.r2_score(y_teste, predictions))
print('MAE:', metrics.mean_absolute_error(y_teste, predictions))
print('MSE:', metrics.mean_squared_error(y_teste, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_teste, predictions)))

In [ ]:
df_clean['TARGET_deathRate'].min(), df_clean['TARGET_deathRate'].max()

In [ ]:
((df_clean["TARGET_deathRate"] - df_clean["TARGET_deathRate"].min())/(df_clean["TARGET_deathRate"].max() - df_clean["TARGET_deathRate"].min())).describe()

In [ ]:
#1) Instanciar o scaler
mms = MinMaxScaler()

#2) Treinar o scaler com os dados de treino
mms.fit(X_treino)

#3) Transformando os dados
X_treino_scaled = mms.transform(X_treino)
X_teste_scaled = mms.transform(X_teste)

In [ ]:
X = df_clean.drop(columns=["TARGET_deathRate"])
y = df_clean['TARGET_deathRate']

# 1)
estimador = LinearRegression()

# 2)
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# NOVO: aplicando o scaler!! - usamos o minmax pra que todas as features
# tenham o mesmo intervalo no espaço amostral - (0, 1)

mms = MinMaxScaler()

# agora, vamos FITAR o scaler COM OS DADOS DE TREINO
mms.fit(X_treino)

# transformando os dados
X_treino = mms.transform(X_treino)
X_teste = mms.transform(X_teste)

X_treino = mms.fit_transform(X_treino)
X_teste = mms.transform(X_teste)


# 3)
modelo = estimador.fit(X_treino, y_treino)

print("Intercepto:", modelo.intercept_)
print("Coeficientes angulares:", modelo.coef_)

print("\n####################################################\n")

# 4)
predictions = modelo.predict(X_teste)

plt.title("valor predito vs valor real")
sns.scatterplot(x=predictions, y=y_teste)
plt.plot(np.linspace(100, 250, 1000), np.linspace(100, 250, 1000), color="red")
plt.show()

plt.title("Distribuição dos resíduos")
sns.histplot(y_teste-predictions, kde=True)
plt.show()

print("\n####################################################\n")

print("\nMétricas de avaliação (dados de teste):\n")

# 5)
from sklearn import metrics

print('R^2:', metrics.r2_score(y_teste, predictions))
print('MAE:', metrics.mean_absolute_error(y_teste, predictions))
print('MSE:', metrics.mean_squared_error(y_teste, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_teste, predictions)))

# CHEGAMOS A UM VEREDITO!

### **Analisando os resultados dos modelos, conclui-se que o modelo de regressão múltipla tem os melhores parâmetros estatísticos de R² do que o de regressão simples.**
